In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import cv2
import os

def embed_message(image, message):
    binary_message = ''.join(format(ord(i), '08b') for i in message)
    binary_message += '1111111111111110'
    binary_message_length = len(binary_message)

    data_index = 0
    for values in image:
        for pixel in values:
            r, g, b = [format(i, '08b') for i in pixel]
            if data_index < binary_message_length:
                pixel[0] = int(r[:-1] + binary_message[data_index], 2)
                data_index += 1
            if data_index < binary_message_length:
                pixel[1] = int(g[:-1] + binary_message[data_index], 2)
                data_index += 1
            if data_index < binary_message_length:
                pixel[2] = int(b[:-1] + binary_message[data_index], 2)
                data_index += 1
            if data_index >= binary_message_length:
                break
    return image

def embed_in_images(input_dir, output_dir, message):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith('.png'):
            image_path = os.path.join(input_dir, filename)
            image = cv2.imread(image_path)
            stego_image = embed_message(image, message)
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, stego_image)
            print(f"Processed {filename}")


input_directory = '/content/drive/MyDrive/Bitirme/LSB/cover_images'
output_directory = '/content/stego_images'
secret_message = "Hicbir sey"
embed_in_images(input_directory, output_directory, secret_message)


Aşağıda, static steganalysis prensiplerine dayalı, ancak derin öğrenme kullanan örnekte, HOG, Color Histograms ve LBP gibi statik özellikler çıkarılır ve bu özellikler kullanılarak bir sinir ağı modeli eğitilir.

In [ ]:
import numpy as np
import cv2
from skimage.feature import hog, local_binary_pattern

def extract_features(image):
    image = cv2.resize(image, (512, 512))
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    hog_features, hog_image = hog(gray_image, pixels_per_cell=(32, 32), cells_per_block=(2, 2), visualize=True, channel_axis=None)
    chans = cv2.split(image)
    colors = ("b", "g", "r")
    features = []

    for (chan, color) in zip(chans, colors):
        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.extend(hist)

    # Local Binary Patterns (LBP)
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(gray_image, n_points, radius, method="uniform")
    (lbp_hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-6)


    combined_features = np.hstack([hog_features, features, lbp_hist])

    return combined_features


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import cv2
import os

def load_data(cover_dir, stego_dir):
    cover_files = sorted([os.path.join(cover_dir, f) for f in os.listdir(cover_dir) if f.endswith('.png')])
    stego_files = sorted([os.path.join(stego_dir, f) for f in os.listdir(stego_dir) if f.endswith('.png')])

    data = []
    labels = []

    for image_path in cover_files:
        image = cv2.imread(image_path)
        features = extract_features(image)
        data.append(features)
        labels.append(0)

    for image_path in stego_files:
        image = cv2.imread(image_path)
        features = extract_features(image)
        data.append(features)
        labels.append(1)

    return np.array(data), np.array(labels)

cover_directory = '/content/drive/MyDrive/Bitirme/LSB/cover_images'
stego_directory = '/content/stego_images'
X, y = load_data(cover_directory, stego_directory)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryCrossentropy(), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

# Modeli kaydetme
model.save('/content/stego_classifier.h5')


In [ ]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
import os

model = load_model('/content/stego_classifier.h5')


def classify_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (512, 512))
    image = image.astype(np.float32) / 255.0
    features = extract_features(image)
    features = scaler.transform([features])

    prediction = model.predict(features)[0][0]
    predicted_label = 'Stego' if prediction >= 0.5 else 'Cover'
    prediction_accuracy = prediction if prediction >= 0.5 else 1 - prediction

    return predicted_label, prediction_accuracy * 100


def calculate_detection_accuracy(test_images_dir):
    test_images = []

    for filename in os.listdir(test_images_dir):
        if filename.endswith('.png'):
            image_path = os.path.join(test_images_dir, filename)
            test_images.append(image_path)

    for i, image_path in enumerate(test_images):
        predicted_label, prediction_accuracy = classify_image(image_path)
        print(f"Image {image_path}, Prediction: {predicted_label}, Prediction accuracy: {prediction_accuracy:.4f}%")

test_images_directory = '/content/drive/MyDrive/Bitirme/LSB/stego_images'

calculate_detection_accuracy(test_images_directory)
